# Prepare for Capital One Technical Interview

## Technical & Design Interview Guidelines

For the **Technical** Interview: Building end-to-end data pipelines with pandas

## Pandas

In [ ]:
from datetime import datetime
import pandas as pd

DATA_FILEPATH = r"../../data/beijing_airquality/PRSA_Data_Changping_20130301-20170228.csv"

# data = pd.read_csv(DATA_FILEPATH, chunksize=10000, header=0, index_col='No', on_bad_lines='warn')
df = pd.read_csv(DATA_FILEPATH, header=0, index_col='No', on_bad_lines='warn')

# display(df)
# df.info()
# df.describe(include='all')
df['year'].value_counts(sort=False).to_dict()

# display(df)


#### De-duping

In [ ]:
print(f"rows before de-dup: {len(df.index)}")
print(f"deduping... ")
df.drop_duplicates(subset=['year', 'month', 'day', 'hour'], inplace=True, ignore_index=True, keep='last')
print(f"rows after de-dup: {len(df.index)}")


#### Basic transforms

In [ ]:
# dropping columns
df = df.drop(columns=['PM2.5', 'SO2', 'NO2', 'O3'], errors='ignore')
# change column names to lower case
[df.rename(columns={col: col.lower()}, inplace=True) for col in list(df.columns) if col.isupper()]
# check columns
necessary_columns = ('year', 'month', 'day', 'hour', 'temp', 'pres', 'dewp', 'rain', 'wd', 'wspm', 'station')
assert all([col in list(df.columns) for col in necessary_columns]), f"Missing schema column"
# use efficient data types
print("data types before cast:")
print(df.dtypes)
df['year'] = pd.to_numeric(df['year'], downcast='unsigned')
df['month'] = pd.to_numeric(df['month'], downcast='unsigned')
df['day'] = pd.to_numeric(df['day'], downcast='unsigned')
df['hour'] = pd.to_numeric(df['hour'], downcast='unsigned')
df['pm10'] = pd.to_numeric(df['pm10'], downcast='float')
df['co'] = pd.to_numeric(df['co'], downcast='float')
df['temp'] = pd.to_numeric(df['temp'], downcast='float')
df['pres'] = pd.to_numeric(df['pres'], downcast='float')
df['dewp'] = pd.to_numeric(df['dewp'], downcast='float')
df['rain'] = pd.to_numeric(df['rain'], downcast='float')
df['wspm'] = pd.to_numeric(df['wspm'], downcast='float')
df['wd'] = df['wd'].astype('category')
df['station'] = df['station'].astype('category')
print("data types after cast:")
print(df.dtypes)
# create date
df['mdate'] = df.apply(lambda row: datetime(year=row['year'], month=row['month'], day=row['day'], hour=row['hour']), axis='columns')
display(df)

#### Detecting and Handling nulls

In [ ]:
# sum of null values
df.isnull().sum()



In [ ]:
# display any rows with nulls
ndf = df[df.isnull().any(axis=1)]
# display(ndf)

# filter columns
ndf = df[['mdate', 'year', 'month', 'day', 'hour', 'temp', 'pres']]

# let's looks only at the null values in temp
ndf = ndf[ndf['temp'].isnull()]
display(ndf)
print(ndf.shape)

interpolate missing temperatures

In [ ]:
# filter columns
xdf = df[['mdate', 'year', 'month', 'day', 'hour', 'temp', 'pres']]

# interpolate
xdf.set_index('mdate', inplace=True)
xdf['temp'] = xdf['temp'].interpolate(method='linear')

# list of missing values
filter_list = list(ndf['mdate'].unique())
xdf = xdf.loc[filter_list]
display(xdf)

using `ffill()` and `bfill()`

In [ ]:
# display(ndf)

xdf = df[['mdate', 'year', 'month', 'day', 'hour', 'temp', 'pres']].copy()
xdf['temp'].ffill(inplace=True)
xdf = xdf[xdf['mdate'].isin(filter_list)]
display(xdf)

#### Analytics

In [ ]:
# average temperature per year

avg_temp = df[['year', 'temp']].groupby(['year']).agg({'temp': 'mean'})
display(avg_temp)


#### Selection and Filtering Exercises

In [ ]:
# filter columns
xdf = df[['year', 'mdate', 'temp', 'rain']]
xdf = xdf[xdf['year'] == 2016]
# display(xdf.iloc[0:10, :2])

# get daily average and show dates that are sub -5 degree
gdf = df[df['year'] == 2016][['year', 'month', 'day', 'temp']]
gdf = gdf.groupby(['year', 'month', 'day']).agg({'temp': 'mean'})
# filter sub -3.0 degree temps
# gdf = gdf[gdf['temp'] < -3.0]
# with query
gdf = gdf.query('temp < -3.0 & ~(year == 2017)')
# gdf.reset_index(inplace=True)
display(gdf)

## Plotting

In [ ]:
import matplotlib.pyplot as plt

# linear plot of temperature
vdf = df[(df['temp'].notnull()) & (df['year'] == 2016) & (df['month'] == 1)]
# vdf = vdf[['year', 'month', 'day', 'hour', 'temp']].set_index(['year', 'month', 'day', 'hour'])
vdf = vdf[['mdate', 'temp']].set_index('mdate')

# df['temp'].plot(kind='line', title='Jan Temps')

plt.plot(vdf.index, vdf['temp'], scalex=True, marker='x')
plt.xticks(rotation=45)
plt.xlabel('Date')
plt.ylabel('Temp')
plt.title('Jan Temperatures')

plt.show()

##### Box plot

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Create a DataFrame
df = pd.DataFrame({
    'category': ['A', 'A', 'B', 'B', 'C', 'C', 'D', 'D'],
    'values': [1, 5, 2, 8, 3, 7, 4, 6]
})

# Create a box plot for the values grouped by category
df.boxplot(column='values', by='category')
plt.title('Box Plot of Values by Category')
plt.suptitle('')
plt.show()

Box chart of wind direction and wind speed to see if wind is stronger in some directions in Beijing!

In [ ]:
import matplotlib.pyplot as plt

# display(df)

# check out null values
tdf = df[(df['wd'].isnull()) | (df['wspm'].isnull())]
null_dates = list(tdf['mdate'])
print(f"wind speed or direction null row count: {tdf.shape[0]}")

# interpolating wind null values
print("filling null values...")
wdf = df.copy()
wdf.set_index('mdate', inplace=True)
wdf['wd'] = wdf['wd'].ffill()
wdf['wspm'] = wdf['wspm'].interpolate(method='linear')
print("showing filled values:")
display(wdf.loc[null_dates][:10])

# print unique wind dirs
display(list(wdf['wd'].unique()))

# box chart of wind direction and speed
df.boxplot(column='wspm', by='wd', fontsize=11)
plt.title("Wind Speed and Direction")
plt.xlabel("wind direction")
plt.ylabel("wind speed (m)")
plt.suptitle("")
plt.show()



##### Heatmap

Heatmap of temperatures in 2016 to see if some temperatures are more prevalent!

In [ ]:
import matplotlib.pylab as plt
import seaborn as sns

# display(df)

# copy data and remove null temperatures
vdf = df.set_index('mdate')[['temp', 'pres']].copy()
# show null counts
print(f"null counts: {vdf.isnull().sum().to_dict()}\t shape: {vdf.shape}")
print("removing nulls...")
vdf = vdf.drop(index=list(vdf[vdf['temp'].isnull()].index))
print(f"null counts: {vdf.isnull().sum().to_dict()}\t shape: {vdf.shape}")

# normalizing temps
print("rounding temperatures...")
vdf['temp'] = vdf['temp'].map(lambda x: round(x, 0))

print("plotting...") 
sns.heatmap(list(vdf['temp']), annot=True, cmap='coolwarm')
plt.title("Beijing Temperature Heatmap")
plt.show()


In [ ]:
import seaborn as sns
import numpy as np

# Create a random correlation matrix
data = np.random.rand(10, 12)
display(data)

# Create a heatmap with Seaborn
sns.heatmap(data, annot=True, cmap='coolwarm')
plt.title('Heatmap of Random Data')
plt.show()
